## Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 

- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 

- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.



## 2) Prepare the data for processing

### Booking

#### We didnt check for duplicates so far... What if the file is has duplicated lines?

In [1]:
%%time
import pandas as pd
bc=pd.read_csv('./bookings.csv.bz2', sep='^', chunksize=500000, dtype=str)
all_chunks=pd.DataFrame()

for i, chunk in enumerate(bc):  
    all_chunks=all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
    print((i+1)*500000,len(all_chunks))
    

all_chunks.to_csv('bookings_no_dup.csv', sep='^', index=False)

500000 500000
1000000 1000000
1500000 1000000
2000000 1000000
2500000 1000000
3000000 1000000
3500000 1000000
4000000 1000000
4500000 1000000
5000000 1000000
5500000 1000003
6000000 1000003
6500000 1000003
7000000 1000003
7500000 1000003
8000000 1000003
8500000 1000003
9000000 1000003
9500000 1000003
10000000 1000003
10500000 1000003
CPU times: user 12min 47s, sys: 18.3 s, total: 13min 6s
Wall time: 12min 48s


In [2]:
%%time
import pandas as pd
bc=pd.read_csv('./searches.csv.bz2', sep='^', chunksize=500000, dtype=str)
all_chunks=pd.DataFrame()

for i, chunk in enumerate(bc):
    print((i)*500000,len(all_chunks))
    all_chunks=all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
    

all_chunks.to_csv('searches_no_dup.csv', sep='^', index=False)

0 0
500000 358999
1000000 358999
1500000 359003
2000000 359003
2500000 359003
3000000 359003
3500000 359003
4000000 359003
4500000 359003
5000000 359003
5500000 359003
6000000 359003
6500000 359003
7000000 359003
7500000 359003
8000000 359003
8500000 359003
9000000 359003
9500000 359003
10000000 359003
10500000 359003
11000000 359003
11500000 359003
12000000 359003
12500000 359003
13000000 359003
13500000 359003
14000000 359003
14500000 359003
15000000 359003
15500000 359003
16000000 359003
16500000 359003
17000000 359003
17500000 359003
18000000 359003
18500000 359003
19000000 359003
19500000 359003
20000000 359003
CPU times: user 17min 16s, sys: 14.8 s, total: 17min 31s
Wall time: 17min 3s


In [5]:
!bzip2 ./bookings_no_dup.csv

In [6]:
!bzip2 ./searches_no_dup.csv

In [7]:
ls -l

total 1081388
-rw-rw-r-- 1 dsc dsc 554970628 may 31 12:52 bookings.csv.bz2
-rw-rw-r-- 1 dsc dsc  55487572 jun  7 19:57 bookings_no_dup.csv.bz2
-rw-rw-r-- 1 dsc dsc   4232732 jun  1 09:41 bookings.sample.csv
-rw-rw-r-- 1 dsc dsc    535893 jun  1 09:24 bookings.sample.csv.bz2
-rw-rw-r-- 1 dsc dsc     15476 jun  1 13:57 ch_01-Empty.ipynb
-rw-rw-r-- 1 dsc dsc     82636 jun  1 13:57 ch_02-Empty.ipynb
-rw-rw-r-- 1 dsc dsc    156925 jun  7 19:31 ch_03-Empty.ipynb
-rw-rw-r-- 1 dsc dsc     11042 jun  7 20:19 ch_04-Empty.ipynb
-rw-rw-r-- 1 dsc dsc     19364 jun  1 13:57 ch_05b-empty.ipynb
-rw-rw-r-- 1 dsc dsc     30965 jun  1 13:57 ch_05-empty.ipynb
-rw-rw-r-- 1 dsc dsc 483188920 may 31 12:52 searches.csv.bz2
-rw-rw-r-- 1 dsc dsc   8504895 jun  7 20:16 searches_no_dup.csv.bz2
-rw-rw-r-- 1 dsc dsc     73325 jun  1 13:07 top_airports.csv


#### We have seen that we have white space in some columns....

In [13]:
import pandas as pd
pd.set_option('display.max_columns', None)
s=pd.read_csv('./searches_no_dup.csv.bz2', sep='^', nrows=1000, usecols=['Date','Origin', 'Destination'])
b=pd.read_csv('./bookings_no_dup.csv.bz2', sep='^', nrows=1000,usecols=['cre_date           ','dep_port','arr_port'])

In [10]:
s.sample()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
359,2013-01-01,21:55:46,MPT,22cf15984f4b13ed8f513a42324666d1,FR,PAR,CLT,1,2,PAR,CLT,2013-02-10,NaN,NaN,CLT,PAR,2013-02-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,PAR


In [11]:
b.sample()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
269,2013-03-05 00:00:00,1A,PK,272127bdb035b5cfe0ec7263b17e2340,207f27ff66f7303dabc7a4c90eda6f6e,e817e9eddcca7e94a854ce35da287983,2013-02-23 00:00:00,20080,0,KHI,KHI,PK,JED,JED,SA,KHI,KHI,PK,JED,JED,SA,KHI,KHI,PK,JEDKHI,JEDKHI,PKSA,1,JEDKHI,DG,H,Y,2013-03-20 01:15:00,2013-03-20 07:25:59,-1,2013,3,NULL


In [12]:
list(b.columns)

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

In [44]:
import pandas as pd
pd.set_option('display.max_columns', None)
s=pd.read_csv('./searches_no_dup.csv.bz2', sep='^',  usecols=['Date','Origin', 'Destination'])
b=pd.read_csv('./bookings_no_dup.csv.bz2', sep='^', usecols=['cre_date           ','dep_port','arr_port'])

In [45]:
b.columns=b.columns.str.strip()

In [46]:
list(b.columns)

['cre_date', 'dep_port', 'arr_port']

In [47]:
b.loc[0]

cre_date    2013-02-22 00:00:00
dep_port               ZRH     
arr_port               LHR     
Name: 0, dtype: object

In [48]:
list(b.loc[0])

['2013-02-22 00:00:00', 'ZRH     ', 'LHR     ']

In [49]:
b.dep_port=b.dep_port.str.strip()

b.arr_port=b.arr_port.str.strip()

In [51]:
list(b.loc[0])

['2013-02-22 00:00:00', 'ZRH', 'LHR']

In [52]:
list(s.loc[0])

['2013-01-01', 'TXL', 'AUH']

In [24]:
b.cre_date[0][:10]

'2013-02-22'

In [53]:
b["cre_date"]=b["cre_date"].str[:10]

In [54]:
list(b.loc[0])

['2013-02-22', 'ZRH', 'LHR']

In [55]:
b.sample()

,cre_date,dep_port,arr_port
542601,2012-11-19,FLN,PVG


In [56]:
s.sample()

,Date,Origin,Destination
39197,2013-02-09,MEX,LIM


###  Search

In [58]:
len(s), len(s.dropna())

(359004, 359003)

In [59]:
len(b), len(b.dropna())

(1000003, 1000003)

In [61]:
s[s.isna().any(axis=1)]

,Date,Origin,Destination
359000,"2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5c...",NaN,NaN


In [65]:
import pandas as pd
pd.set_option('display.max_columns', None)
s=pd.read_csv('./searches_no_dup.csv.bz2', sep='^',  usecols=['Date','Origin', 'Destination'])
b=pd.read_csv('./bookings_no_dup.csv.bz2', sep='^', usecols=['cre_date           ','dep_port','arr_port'])
b.columns=b.columns.str.strip()
b.dep_port=b.dep_port.str.strip()
b.arr_port=b.arr_port.str.strip()
b["cre_date"]=b["cre_date"].str[:10]
b=b.dropna()
s=s.dropna()

In [66]:
s=s.reset_index()

In [67]:
s.tail()

,index,Date,Origin,Destination
358998,358998,2013-12-25,LAX,TLV
358999,358999,2013-01-01,TXL,AUH
359000,359001,2013-01-01,ICT,SFO
359001,359002,2013-01-01,FRA,BGW
359002,359003,2013-10-13,VIE,HA


## 3) Make action plan

In [ ]:
drop duplicates en bookings
Search left join


In [78]:
b_no_dup=b.groupby(['cre_date','dep_port','arr_port']).size().reset_index()

b_no_dup.rename(columns={0:'NumBooks'}, inplace=True)

In [81]:
b_no_dup.head()

,cre_date,dep_port,arr_port,NumBooks
0,2011-03-01,CDG,WNZ,4
1,2011-03-01,LIS,DKR,2
2,2011-03-26,MRS,HAH,2
3,2011-06-28,BKK,CNX,2
4,2011-06-28,SYD,LHR,3


In [82]:
s.sample()

,index,Date,Origin,Destination
195703,195703,2013-07-15,CUN,STR


In [88]:
s_b=s.merge(b_no_dup,
            how='left',
           left_on=['Date', 'Origin', 'Destination'],
           right_on=['cre_date', 'dep_port', 'arr_port'])

In [85]:
s_b.sample()

,index,Date,Origin,Destination,cre_date,dep_port,arr_port,NumBooks
208064,208064,2013-07-28,LTN,BCN,NaN,NaN,NaN,NaN


In [86]:
s_b.tail()

,index,Date,Origin,Destination,cre_date,dep_port,arr_port,NumBooks
358998,358998,2013-12-25,LAX,TLV,NaN,NaN,NaN,NaN
358999,358999,2013-01-01,TXL,AUH,NaN,NaN,NaN,NaN
359000,359001,2013-01-01,ICT,SFO,NaN,NaN,NaN,NaN
359001,359002,2013-01-01,FRA,BGW,NaN,NaN,NaN,NaN
359002,359003,2013-10-13,VIE,HA,NaN,NaN,NaN,NaN


In [89]:
del s_b['cre_date']
del s_b['dep_port']
del s_b['arr_port']

In [92]:
s_b[s_b["NumBooks"]>0].head()

,index,Date,Origin,Destination,NumBooks
27,27,2013-01-01,RUH,JED,17.0
40,40,2013-01-01,DMM,MNL,5.0
59,59,2013-01-01,ATL,MIA,6.0
134,134,2013-01-01,MEL,SYD,10.0
172,172,2013-01-01,BOM,JED,2.0


In [93]:
s_b["Booked"]=s_b["NumBooks"].fillna(0).map(lambda x: 1 if x>0 else 0)

In [95]:
s_b.sample()

,index,Date,Origin,Destination,NumBooks,Booked
74242,74242,2013-03-16,YYZ,SAN,NaN,0


In [96]:
s_b[s_b["NumBooks"]>0].sample()

,index,Date,Origin,Destination,NumBooks,Booked
34226,34226,2013-02-04,MAD,BCN,1.0,1


In [98]:
s_all=pd.read_csv('./searches_no_dup.csv.bz2', sep='^', low_memory=False)

results=s_all.merge(s_b[["index", "Booked"]],
                    how='left',
                    left_index=True,
                    right_on="index"
                   )

In [101]:
results.tail()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,index,Booked
358999,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,NaN,NaN,NaN,358999,0.0
359002,"2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,359000,NaN
359000,2013-01-01,18:04:49,MPT,3561,US,ICT,SFO,1.0,2.0,ICT,SFO,2013-08-02,NaN,NaN,SFO,ICT,2013-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,359001,0.0
359001,2013-01-01,19:57:57,MPT,28d7a8c95e4db88589d3d35b66920e78,DE,FRA,BGW,1.0,2.0,FRA,BGW,2013-02-26,NaN,NaN,BGW,FRA,2013-04-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,BNJ,NaN,NaN,359002,0.0
359002,2013-10-13,18:57:54,MTP,e41c9d833aa74600552f2ed688b67d81,AT,VIE,HA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,359003,0.0


In [102]:
del results['index']

In [103]:
results.to_csv('Done', sep='^', index=False)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
s=pd.read_csv('./searches_no_dup.csv.bz2', sep='^',  usecols=['Date','Origin', 'Destination'])
b=pd.read_csv('./bookings_no_dup.csv.bz2', sep='^', usecols=['cre_date           ','dep_port','arr_port'])
b.columns=b.columns.str.strip()
b.dep_port=b.dep_port.str.strip()
b.arr_port=b.arr_port.str.strip()
b["cre_date"]=b["cre_date"].str[:10]
b=b.dropna()
s=s.dropna()
s=s.reset_index()
b_no_dup=b.groupby(['cre_date','dep_port','arr_port']).size().reset_index()

b_no_dup.rename(columns={0:'NumBooks'}, inplace=True)

s_b=s.merge(b_no_dup,
            how='left',
           left_on=['Date', 'Origin', 'Destination'],
           right_on=['cre_date', 'dep_port', 'arr_port'])
s_b["Booked"]=s_b["NumBooks"].fillna(0).map(lambda x: 1 if x>0 else 0)

s_all=pd.read_csv('./searches_no_dup.csv.bz2', sep='^', low_memory=False)

results=s_all.merge(s_b[["index", "Booked"]],
                    how='left',
                    left_index=True,
                    right_on="index"
                   )
del results['index']
results.to_csv('Done', sep='^', index=False)